In [ ]:
ANNOTATE = True

import numpy as np
import holoviews as hv
from bokeh.models import HoverTool
from holoviews import Dataset
from holoviews.plotting.links import RangeToolLink
from scipy.stats import zscore

hv.extension('bokeh')

N_CHANNELS = 10
N_SECONDS = 5
SAMPLING_RATE = 200
INIT_FREQ = 2  # Initial frequency in Hz
FREQ_INC = 5  # Frequency increment
AMPLITUDE = 1

# Generate time and channel labels
total_samples = N_SECONDS * SAMPLING_RATE
time = np.linspace(0, N_SECONDS, total_samples)
channels = [f'EEG {i}' for i in range(N_CHANNELS)]

# Generate sine wave data
data = np.array([AMPLITUDE * np.sin(2 * np.pi * (INIT_FREQ + i * FREQ_INC) * time)
                     for i in range(N_CHANNELS)])

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")
])

annotation = hv.VSpan(1, 1.5).opts(color='yellow', alpha=.15) # example annotation (start, end) time

channel_curves = []
for channel, channel_data in zip(channels, data):
    ds = Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=f'{channel}')
    curve.opts(color="black", line_width=1, subcoordinate_y=True, tools=[hover])
    channel_curves.append(curve)

if ANNOTATE:
    eeg_viewer = (annotation * hv.Overlay(channel_curves, kdims="Channel"))
else:
    eeg_viewer = hv.Overlay(channel_curves, kdims="Channel")
eeg_viewer = eeg_viewer.opts(padding=0,
    xlabel="Time (s)", ylabel="Channel", show_legend=False, aspect=3, responsive=True,
)

y_positions = range(N_CHANNELS)
yticks = [(i , ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

minimap = hv.Image((time, y_positions , z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", xlabel='Time (s)', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=150, responsive=True, default_tools=[], clim=(-z_data.std(), z_data.std())
)

RangeToolLink(
    minimap, eeg_viewer, axes=["x", "y"],
    boundsx=(None, 2), boundsy=(None, 6.5)
)
if ANNOTATE:
    dashboard = (eeg_viewer + minimap * annotation).opts(merge_tools=False).cols(1)
else:
    dashboard = (eeg_viewer + minimap).opts(merge_tools=False).cols(1)
dashboard